## Requisitos

- Cuenta y deployment de **Azure OpenAI** (modelo, p. ej. `gpt-4o-mini`).
- Variables de entorno:
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_DEPLOYMENT_NAME` (opcional, por defecto `gpt-4o-mini`)
- Paquetes: `agent-framework`, `agent-framework-azurefunctions`, `azure-identity`, `pandas`

Nota: Seguimos usando CSV/JSON en `data/` y `out/` como en LAB 2 para persistencia sencilla de tickets.

In [ ]:
# Instalación de dependencias (ejecuta una vez en tu entorno)
%pip install -q agent-framework agent-framework-azurefunctions azure-identity pandas python-dotenv

In [ ]:
# Cargar configuración Azure OpenAI desde .env o entorno
import os
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME', 'gpt-4o-mini')

if not AZURE_OPENAI_ENDPOINT:
    raise RuntimeError('❌ Falta la variable AZURE_OPENAI_ENDPOINT.')

print('✅ Azure OpenAI configurado')
print('  ENDPOINT:', AZURE_OPENAI_ENDPOINT)
print('  DEPLOYMENT:', AZURE_OPENAI_DEPLOYMENT_NAME)

In [ ]:
# Carpetas y utilidades de datos (igual filosofía que LAB 2)
from pathlib import Path
import json
import pandas as pd
from datetime import datetime

DATA_DIR = Path('data')
OUT_DIR = Path('out')
DATA_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

TICKETS_CSV = DATA_DIR / 'tickets.csv'
if not TICKETS_CSV.exists():
    df_init = pd.DataFrame(columns=["id", "fecha", "solicitante", "departamento", "categoria", "prioridad", "estado", "resumen", "detalle"])
    df_init.to_csv(TICKETS_CSV, index=False, encoding='utf-8')

def load_tickets() -> pd.DataFrame:
    if TICKETS_CSV.exists():
        return pd.read_csv(TICKETS_CSV, dtype=str).fillna('')
    return pd.DataFrame(columns=["id", "fecha", "solicitante", "departamento", "categoria", "prioridad", "estado", "resumen", "detalle"])

In [ ]:
# Tools: crear y actualizar tickets (compatibles con los LABs previos)
from typing import Dict, Annotated
from pydantic import Field

def crear_ticket_fc(
    desde_email: Annotated[str, Field(description='Correo del solicitante')],
    departamento: Annotated[str, Field(description='IT, RRHH, Facilities u otro')],
    categoria: Annotated[str, Field(description='nuevo_equipo, incidencia, vacaciones, etc.')],
    prioridad: Annotated[str, Field(description='alta, media, baja')],
    resumen: Annotated[str, Field(description='Resumen corto')],
    detalle: Annotated[str, Field(description='Detalle extendido')],
) -> Dict:
    data = load_tickets()
    next_id = 1 if data.empty else int(pd.to_numeric(data['id'], errors='coerce').max() or 0) + 1
    nuevo = {
        'id': str(next_id),
        'fecha': datetime.now().strftime('%Y-%m-%d'),
        'solicitante': desde_email,
        'departamento': departamento or 'Otro',
        'categoria': categoria or 'otro',
        'prioridad': prioridad or 'media',
        'estado': 'pendiente',
        'resumen': (resumen or '')[:200],
        'detalle': detalle or '',
    }
    data = pd.concat([data, pd.DataFrame([nuevo])], ignore_index=True)
    data.to_csv(TICKETS_CSV, index=False, encoding='utf-8')
    with (OUT_DIR / f'ticket_{next_id}.json').open('w', encoding='utf-8') as f:
        json.dump(nuevo, f, ensure_ascii=False, indent=2)
    return nuevo

def actualizar_estado_ticket_fc(
    id_ticket: Annotated[int, Field(description='Id del ticket a actualizar')],
    nuevo_estado: Annotated[str, Field(description='pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado')],
) -> Dict:
    data = load_tickets()
    if data.empty:
        return {'ok': False, 'mensaje': 'No hay tickets.'}
    mask = data['id'] == str(id_ticket)
    if not mask.any():
        return {'ok': False, 'mensaje': f'No existe el ticket {id_ticket}'}
    data.loc[mask, 'estado'] = nuevo_estado
    data.to_csv(TICKETS_CSV, index=False, encoding='utf-8')
    actualizado = data.loc[mask].iloc[0].to_dict()
    # Espejar al JSON individual si existe
    p = OUT_DIR / f'ticket_{id_ticket}.json'
    if p.exists():
        with p.open('w', encoding='utf-8') as f:
            json.dump(actualizado, f, ensure_ascii=False, indent=2)
    return {'ok': True, 'ticket': actualizado}

## Agente Durable (Azure OpenAI) + AgentFunctionApp

A continuación creamos un **agente duradero** con Azure OpenAI y registramos tools.
Con `AgentFunctionApp(agents=[agent])` se habilita hosting serverless (Azure Functions) con **gestion de estado durable** y endpoints HTTP automáticos.
En este notebook lo configuramos; el despliegue/hosting real se hace en un proyecto de Azure Functions.

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient, AgentFunctionApp
from azure.identity import DefaultAzureCredential
from agent_framework import ChatAgent

SERVICE_DESK_INSTRUCTIONS = """
Eres un agente de Service Desk interno.
- Entiende la solicitud (vacaciones, incidencias IT, facilities...).
- Usa herramientas cuando debas crear/actualizar tickets.
- Responde claro y en español. No devuelvas JSON al usuario.
Si falta algún dato crítico (p. ej., email o id de ticket), pídelo.
"""

# Cliente Azure OpenAI
chat_client = AzureOpenAIChatClient(
    endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    credential=DefaultAzureCredential(),
)

# Agente duradero (con tools)
durable_agent = chat_client.create_agent(
    name='DurableServiceDeskAgent',
    instructions=SERVICE_DESK_INSTRUCTIONS,
    tools=[crear_ticket_fc, actualizar_estado_ticket_fc],
)

# App de funciones (serverless) que gestiona estado durable y expone endpoints
app = AgentFunctionApp(agents=[durable_agent])
print('✅ Agente duradero configurado. Listo para ser alojado con Azure Functions.')

### Hosting (resumen)

- En un proyecto de Azure Functions, coloca este código como arranque (ej. `__init__.py`/`function_app.py`).
- Instala paquetes: `agent-framework-azurefunctions`, `azure-identity`.
- Configura `AZURE_OPENAI_ENDPOINT` y `AZURE_OPENAI_DEPLOYMENT_NAME` como settings.
- Al ejecutarse, se crean endpoints HTTP que gestionan **threads/conversaciones durables**.

## Conversación Durable (local) – Serializar/Deserializar Thread

Si no vas a desplegar todavía, puedes simular la durabilidad **serializando** un thread y **restaurándolo** luego para continuar la conversación.

In [ ]:
import asyncio
from pathlib import Path

THREAD_SNAPSHOT = Path('out/durable_thread.json')

async def demo_durable_thread_local():
    agent = durable_agent  # reutilizamos el agente creado arriba
    # Crear un thread nuevo o restaurar si existe
    if THREAD_SNAPSHOT.exists():
        # Restaurar
        serialized = THREAD_SNAPSHOT.read_text(encoding='utf-8')
        thread = await agent.deserialize_thread(serialized)
        print('♻️ Thread restaurado desde snapshot.')
    else:
        thread = agent.get_new_thread()
        print('🆕 Thread nuevo creado.')

    # Turno 1
    r1 = await agent.run('Mi email es ana.garcia@empresa.local y necesito un portátil nuevo.', thread=thread)
    print('
🤖 Respuesta 1:
', r1.text)

    # Guardar snapshot del thread (durabilidad básica)
    serialized = await thread.serialize()
    THREAD_SNAPSHOT.write_text(serialized, encoding='utf-8')
    print('
💾 Snapshot guardado en', THREAD_SNAPSHOT)

    # Más tarde... continuar conversación restaurando el mismo thread
    new_thread = await agent.deserialize_thread(serialized)
    r2 = await agent.run('Puedes actualizar el ticket a en_progreso si ya está asignado?', thread=new_thread)
    print('
🤖 Respuesta 2:
', r2.text)

await demo_durable_thread_local()